# Context Manager

In [1]:
#cty.py

from sqlite3 import connect

# with open('ctx.py') as f:
#     pass

# Looks like the following behind the scene
# x = ctx().__enter__
# try:
#     pass
# finally:
#     x.__exit__

In [2]:
with connect('test.db') as conn:
    cur = conn.cursor()
    cur.execute('create table points(x int, y int)')
    cur.execute('insert into points (x, y) values(1,1)')
    cur.execute('insert into points (x, y) values(1,2)')
    cur.execute('insert into points (x, y) values(2,1)')
    for row in cur.execute('select x, y from points'):
        print(row)
    for row in cur.execute('select sum(x * y) from points'):
        print(row)

    cur.execute('drop table points')

(1, 1)
(1, 2)
(2, 1)
(5,)


Context manager is a mechanism allows acquisition and release of resources through the implementation of **\__enter\__** and **\__exit\__** functions in a class

In [13]:
class temptable:
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        print('__enter__')
        self.cur.execute('create table points(x int, y int)')
    def __exit__(self, *args):
        print('__exit__')
        self.cur.execute('drop table points')

with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values(1,1)')
        cur.execute('insert into points (x, y) values(1,2)')
        cur.execute('insert into points (x, y) values(2,1)')
        for row in cur.execute('select x, y from points'):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)

__enter__
(1, 1)
(1, 2)
(2, 1)
(5,)
__exit__


Since the functions of the Context Manager have to be executed in sequence, **\__enter\__** before **\__exit\__**,  we can create a **Generator** to enforce the behaivour. 

In [14]:
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    print('created table')
    yield
    cur.execute('drop table points')
    print('dropped table')
    
class contextmanager:
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        self.gen = temptable(self.cur)
        next(self.gen)
    def __exit__(self, *args):
        next(self.gen, None)
        
with connect('test.db') as conn:
    cur = conn.cursor()
    with contextmanager(cur):
        cur.execute('insert into points (x, y) values(1, 1)')
        cur.execute('insert into points (x, y) values(1, 2)')
        cur.execute('insert into points (x, y) values(2, 1)')
        cur.execute('insert into points (x, y) values(2, 2)')
        for row in cur.execute("select x, y from points"):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)

created table
(1, 1)
(1, 2)
(2, 1)
(2, 2)
(9,)
dropped table


The **contextmanager** class is too specific, arguments are hard coded, we need to generalise it by doing the following ....

In [15]:
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    print('created table')
    yield
    cur.execute('drop table points')
    print('dropped table')

    
class contextmanager:
    def __init__(self, gen):
        self.gen = gen
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)

with connect('test.db') as conn:
    cur = conn.cursor()
    with contextmanager(temptable)(cur):
        cur.execute('insert into points (x, y) values(1, 1)')
        cur.execute('insert into points (x, y) values(1, 2)')
        cur.execute('insert into points (x, y) values(2, 1)')
        cur.execute('insert into points (x, y) values(2, 2)')
        for row in cur.execute("select x, y from points"):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)


created table
(1, 1)
(1, 2)
(2, 1)
(2, 2)
(9,)
dropped table


**with contextmanager(temptable)(cur)** <br/>
line looks ugly and not easily understood. So we can do the following ....

In [16]:
class contextmanager:
    def __init__(self, gen):
        self.gen = gen
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)

def temptable(cur):
    cur.execute('create table points(x int, y int)')
    print('created table')
    yield
    cur.execute('drop table points')
    print('dropped table')
 
# This looks farmiliar
temptable = contextmanager(temptable)

with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values(1, 1)')
        cur.execute('insert into points (x, y) values(1, 2)')
        cur.execute('insert into points (x, y) values(2, 1)')
        cur.execute('insert into points (x, y) values(2, 2)')
        for row in cur.execute("select x, y from points"):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)

created table
(1, 1)
(1, 2)
(2, 1)
(2, 2)
(9,)
dropped table


**temptable = contextmanager(temptable)** <br/>
line is a **Decorator** so let us do the right thing ....

In [17]:
class contextmanager:
    def __init__(self, gen):
        self.gen = gen
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)

@contextmanager        
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    print('created table')
    yield
    cur.execute('drop table points')
    print('dropped table')

with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values(1, 1)')
        cur.execute('insert into points (x, y) values(1, 2)')
        cur.execute('insert into points (x, y) values(2, 1)')
        cur.execute('insert into points (x, y) values(2, 2)')
        for row in cur.execute("select x, y from points"):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)

created table
(1, 1)
(1, 2)
(2, 1)
(2, 2)
(9,)
dropped table


Obviously the python gurus have implemented the **contextmanager** code as a library, its too generic not be be implemented.

In [12]:
from sqlite3 import connect
from contextlib import contextmanager

@contextmanager        
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    print('created table')
    try:
        yield
    finally:
        cur.execute('drop table points')
        print('dropped table')

with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values(1, 1)')
        cur.execute('insert into points (x, y) values(1, 2)')
        cur.execute('insert into points (x, y) values(2, 1)')
        cur.execute('insert into points (x, y) values(2, 2)')
        for row in cur.execute("select x, y from points"):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)

created table
(1, 1)
(1, 2)
(2, 1)
(2, 2)
(9,)
dropped table


The Final Code. 

**This code uses Context Managers --> Generators --> Decorators --> Classes**